In [1]:
import pystan
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
sns.set(rc={'figure.figsize':(14, 10)})

## 16.5

In [3]:
Y = [9, 7, 11, 10, 10, 9, 8, 11, 9, 11]
assert len(Y) == 10

### 16.5.1

$ Y[i] \sim \mathcal{B}(N, \theta)$


### 16.5.2

$ Y[i] \sim \mathcal{B}(N, \theta)$

$ \theta \sim beta(7, 2)$

$ N \sim uniform(11, 20) $

In [4]:
'''
data {
    int<lower=0> nExperiments;
    int<lower=0, upper=20> Y[nExperiments];
}

model {
    theta ~ beta(7, 2); 
    matrix[nExperiments, 10] logp;
    for(e in 1:nExperiments){
        for(n in 1:10){
        logp[e, n] = log(0.1) + binomial_lpmf(Y[e] | n+10, theta);
    }
}
'''

'\ndata {\n    int<lower=0> nExperiments;\n    int<lower=0, upper=20> Y[nExperiments];\n}\n\nmodel {\n    theta ~ beta(7, 2); \n    matrix[nExperiments, 10] logp;\n    for(e in 1:nExperiments){\n        for(n in 1:10){\n        logp[e, n] = log(0.1) + binomial_lpmf(Y[e] | n+10, theta);\n    }\n}\n'

### 16.5.3

todo

### 16.5.4

In [5]:
ocode = """
transformed parameters {
    matrix[nExperiments, 10] logp;
    for(e in 1:nExperiments){
        for(n in 1:10){
        logp[e, n] = log(0.1) + binomial_lpmf(Y[e] | n+10, theta);
    }
}
"""

### 16.5.5

In [44]:
data = {'nExperiments': len(Y), 'Y': Y}

In [41]:
ocode = """
data {
    int<lower=1> nExperiments;
    int<lower=0, upper=20> Y[nExperiments];
}

parameters {
    real<lower=0, upper=1> theta;
}


transformed parameters {
    vector[10] lp;
    for(s in 1:10){
        lp[s] = log(0.1) + binomial_lpmf(Y | s+10, theta);
    }
}

model {
    theta ~ beta(7, 2);
    target += log_sum_exp(lp);
}

generated quantities {
    simplex[10] pState;
    pState = exp(lp - log_sum_exp(lp));
}
"""
sm = pystan.StanModel(model_code=ocode)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_146055d62ad978c4d7685f0248a2c14a NOW.


In [45]:
fit = sm.sampling(data=data, iter=1000, chains=4, seed=1)
fit

Inference for Stan model: anon_model_146055d62ad978c4d7685f0248a2c14a.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd    2.5%     25%     50%     75%  97.5%  n_eff   Rhat
theta        0.79  3.5e-3   0.08    0.58    0.74    0.81    0.85    0.9    582    1.0
lp[1]      -22.13    0.24   5.67  -39.68  -23.23  -19.73  -18.66  -18.4    561    1.0
lp[2]      -21.27    0.13    3.4  -31.21  -22.07  -20.04  -19.13 -18.82    635   1.01
lp[3]      -23.76    0.16   4.53  -34.89  -26.26  -22.37  -20.05 -19.33    811    1.0
lp[4]       -28.3    0.28   7.67  -46.02  -33.11  -26.58  -21.74 -19.77    729    1.0
lp[5]       -34.3    0.43  11.26   -58.9  -41.94  -32.25  -24.66 -20.19    689    1.0
lp[6]      -41.41    0.58  14.97  -72.89  -51.88  -39.46  -28.86 -20.65    670    1.0
lp[7]      -49.38    0.73  18.73  -87.75  -62.69  -47.55  -34.11 -21.02    659    1.0
lp[8]      -58.07    0.88  22.52  -103.